In [1]:
import pandas as pd
import spacy

In [2]:
dados_treino = pd.read_csv("../Data/resultado_drogas.csv", encoding = 'latin-1', sep = ';',dtype='unicode')
dados_treino.sample(5)

,CHAVE_OBJETO,ANO_OCOR,NRO_OCOR,NRO_INT_TIPO_OBJETO,TXT_TIPO_OBJETO,Cor Predominante,Descrição,Peso Total,Peso Unitário,Quantidade de Unidade(s),Tipo de Embalagem,Unidade de Peso,DESC_PADRAO,RECLASSIFICAÇÃO
4664,21094581,2019,228,1025,Drogas,Amarelo,Buck Stack 350 vinculado ao codigo OA112488549BR,NaN,NaN,1,Frasco(s),NaN,buck stack 350 vinculado ao codigo oa112488549br,Drogas
26629,23075271,2020,274,1025,Drogas,Verde,1 PORÇÃO DE ERVA ESVERDEADA SEMELHANTE MACONHA,NaN,"13,00",1,Porção(es),Grama,1 porcao de erva esverdeada semelhante maconha,Maconha
85529,23061886,2020,3645,756,Cocaína,Branco,"10 PORÇÕES DE COCAÍNA, TOTALIZANDO 35 GRAMAS","35,00",NaN,10,Porção(es),Grama,10 porcoes de cocaina totalizando 35 gramas,Cocaína
104727,23803633,2021,4429,757,Crack,Marrom,"13,10 GRAMAS DE SUBSTANCIA SEMELHANTE A CRACK",NaN,NaN,0,Porção(es),Grama,1310 gramas de substancia semelhante a crack,Crack
64349,22295666,2020,2864,756,Cocaína,Cinza,252 pinos de substância semelhante a cocaína p...,"170,00","0,07",252,Unidade(s),Grama,252 pinos de substancia semelhante a cocaina p...,Cocaína


In [3]:
nlp = spacy.load('pt_core_news_lg')

In [50]:
def trata_textos(doc):
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text)

    if len(tokens_validos) > 2:
        return  " ".join(tokens_validos)


texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa de Deus!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa Deus'

In [5]:
textos_para_tratamento = (str(descricoes).lower() for descricoes in dados_treino["Descrição"])
textos_para_tratamento

<generator object <genexpr> at 0x000002A74FC41D60>

In [9]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                        batch_size = 120000,
                                                        n_process = -1)]

tf = time() - t0

print(tf/60)

7.065493321418762


In [10]:
titulos_tratados = pd.DataFrame({"descricao": textos_tratados})
titulos_tratados.sample(20)

,descricao
3396,apreendido cigarros substância semelhante maco...
68479,None
111628,porções substância semelhante crack pesando ap...
80567,bucha pó branco semelhante cocaina peso gr
23468,substãncia semelhante cocaína
32472,None
65254,aproximadamente gramas maconha
52758,gramas substância semelhante maconha
84843,tijolo maconha pesando gramas aproximadamente
2386,pinos pequenos substancia semelhante cocaina


In [11]:
from gensim.models import Word2Vec

In [12]:
#sg = pois queremos o modelo cbow e não skipGram
#window = quantas palavras ele considera antes e quantas depois
#vector_size= dimenões do meu vetor
#min_count = minimo de palavras presentes
#Valor que vai determinar qual a taxa que estou aprendendo e reduzindo minha função de custo
#Min alpha o valor da distancia que a função de custo vai decair
w2v_modelo = Word2Vec(sg = 0,
                      window = 2,
                      vector_size = 300,
                      min_count = 0,
                      alpha = 0.03,
                      min_alpha = 0.007)

In [13]:
w2v_modeblo

In [14]:
print(len(titulos_tratados))

titulos_tratados = titulos_tratados.dropna().drop_duplicates()

print(len(titulos_tratados))

113231
51280


In [15]:
lista_lista_tokens = [desc.split(' ') for desc in titulos_tratados.descricao]

In [16]:
w2v_modelo.build_vocab(lista_lista_tokens)

In [17]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg = 0,
                      window = 5,
                      vector_size = 300,
                      min_count = 0,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per = 5000)

2021-09-30 15:15:38,979 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2021-09-30T15:15:38.979372', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2021-09-30 15:15:38,981 : - collecting all words and their counts
2021-09-30 15:15:38,982 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 15:15:38,997 : - PROGRESS: at sentence #5000, processed 29277 words, keeping 1822 word types
2021-09-30 15:15:39,011 : - PROGRESS: at sentence #10000, processed 60106 words, keeping 2813 word types
2021-09-30 15:15:39,022 : - PROGRESS: at sentence #15000, processed 92131 words, keeping 3740 word types
2021-09-30 15:15:39,035 : - PROGRESS: at sentence #20000, processed 125251 words, keeping 4473 word types
2021-09-30 15:15:39,051 : - PROGRESS: at sentence #25000, processed 155609 w

In [18]:
dir(w2v_modelo)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [19]:
w2v_modelo.corpus_count

51280

In [20]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs=30)

2021-09-30 15:15:50,290 : - Word2Vec lifecycle event {'msg': 'training model with 3 workers on 8018 vocabulary and 300 features, using sg=0 hs=0 sample=0.001 negative=5 window=5 shrink_windows=True', 'datetime': '2021-09-30T15:15:50.290624', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'train'}
2021-09-30 15:15:50,852 : - worker thread finished; awaiting finish of 2 more threads
2021-09-30 15:15:50,877 : - worker thread finished; awaiting finish of 1 more threads
2021-09-30 15:15:50,888 : - worker thread finished; awaiting finish of 0 more threads
2021-09-30 15:15:50,889 : - EPOCH - 1 : training on 330737 raw words (168114 effective words) took 0.5s, 314193 effective words/s
2021-09-30 15:15:51,464 : - worker thread finished; awaiting finish of 2 more threads
2021-09-30 15:15:51,476 : - worker thread finished; awaiting finish of 1 more threads
2021-09-30 1

(5048681, 9922110)

In [21]:
w2v_modelo.wv.most_similar("cocaina")

[('cocaína', 0.7129589319229126),
 ('cocaíca', 0.4762324094772339),
 ('cocaiina', 0.4380021393299103),
 ('cocaíana', 0.436846524477005),
 ('eppenford', 0.42361265420913696),
 ('cocacina', 0.40337687730789185),
 ('cacaína', 0.3977382183074951),
 ('cacaina', 0.37509796023368835),
 ('cocaine', 0.3686312735080719),
 ('côr', 0.3666975200176239)]

In [22]:
w2v_modelo.wv.most_similar("maconha")

[('prensada', 0.44905149936676025),
 ('artesanais', 0.44679534435272217),
 ('erva', 0.44316309690475464),
 ('maconhaha', 0.4345993399620056),
 ('maconhaa', 0.43023690581321716),
 ('maconhanha', 0.4228149354457855),
 ('macona', 0.4175446331501007),
 ('esverdeada', 0.4125462472438812),
 ('machonha', 0.41027700901031494),
 ('amaconha', 0.4097497761249542)]

In [23]:
w2v_modelo.wv.most_similar("crak")

[('crack', 0.7438182830810547),
 ('carck', 0.5109866261482239),
 ('ckack', 0.5034031271934509),
 ('crck', 0.4934164583683014),
 ('ptotalizando', 0.42175883054733276),
 ('inteira', 0.4157085418701172),
 ('papela', 0.4139401316642761),
 ('vintee', 0.40566036105155945),
 ('crakc', 0.40253958106040955),
 ('pap', 0.40126633644104004)]

In [24]:
import logging

w2v_modelo_sg = Word2Vec(sg = 1,
                      window = 5,
                      vector_size = 300,
                      min_count = 0,
                      alpha = 0.03,
                      min_alpha = 0.007)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per = 5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs=30)

2021-09-30 15:16:28,919 : - Word2Vec lifecycle event {'params': 'Word2Vec(vocab=0, vector_size=300, alpha=0.03)', 'datetime': '2021-09-30T15:16:28.919095', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'created'}
2021-09-30 15:16:28,921 : - collecting all words and their counts
2021-09-30 15:16:28,922 : - PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-09-30 15:16:28,933 : - PROGRESS: at sentence #5000, processed 29277 words, keeping 1822 word types
2021-09-30 15:16:28,941 : - PROGRESS: at sentence #10000, processed 60106 words, keeping 2813 word types
2021-09-30 15:16:28,955 : - PROGRESS: at sentence #15000, processed 92131 words, keeping 3740 word types
2021-09-30 15:16:28,968 : - PROGRESS: at sentence #20000, processed 125251 words, keeping 4473 word types
2021-09-30 15:16:28,979 : - PROGRESS: at sentence #25000, processed 155609 w

(5049557, 9922110)

In [25]:
w2v_modelo_sg.wv.most_similar("maconha")

[('maconhapesando', 0.5429049730300903),
 ('amaconha', 0.5263329744338989),
 ('macionha', 0.5222907066345215),
 ('machonha', 0.5155734419822693),
 ('macona', 0.5114761590957642),
 ('erva', 0.5063655376434326),
 ('maoonha', 0.5000309944152832),
 ('maconhaa', 0.4965268671512604),
 ('maconnha', 0.49501457810401917),
 ('maconhanha', 0.494768351316452)]

In [26]:
w2v_modelo_sg.wv.most_similar("crak")

[('crack', 0.6502164602279663),
 ('ckack', 0.5305534601211548),
 ('gramns', 0.4862755537033081),
 ('cloração', 0.46918985247612),
 ('crcak', 0.4633367657661438),
 ('porcionadas', 0.4627552628517151),
 ('carck', 0.4623532295227051),
 ('grandede', 0.4570107161998749),
 ('fracionandas', 0.45640286803245544),
 ('suelem', 0.4558532238006592)]

In [27]:
w2v_modelo_sg.wv.most_similar("cocaina")

[('cocaína', 0.6742475032806396),
 ('cocacina', 0.6040804386138916),
 ('cocaíana', 0.6026768684387207),
 ('cocina', 0.6005477905273438),
 ('cacaina', 0.5982316732406616),
 ('aspesto', 0.5962383151054382),
 ('cocaí', 0.5914677381515503),
 ('cocoaina', 0.5749762058258057),
 ('cociana', 0.573636531829834),
 ('cocainaa', 0.5570421814918518)]

In [28]:
w2v_modelo_sg.wv.most_similar("cacaina")

[('subtsancia', 0.6784822940826416),
 ('aproximadamaente', 0.6469683647155762),
 ('àcrak', 0.6429532170295715),
 ('emmbaldas', 0.6352480053901672),
 ('tipica', 0.6292688846588135),
 ('cocina', 0.6280522346496582),
 ('duma', 0.6228322982788086),
 ('evolucron', 0.6214878559112549),
 ('totaldo', 0.6181179285049438),
 ('cocacina', 0.6118002533912659)]

In [29]:
w2v_modelo_sg.wv.most_similar("àcrak")

[('cocainda', 0.9614291787147522),
 ('aproximdamante', 0.9243427515029907),
 ('subtsancia', 0.9156140685081482),
 ('denominadas', 0.9022440910339355),
 ('esmarelada', 0.9008833169937134),
 ('aproximadamented', 0.8714464902877808),
 ('emmbaldas', 0.8662444949150085),
 ('aproximadamentd', 0.8659058809280396),
 ('cigarritos', 0.8566253185272217),
 ('aproximadamaente', 0.8495968580245972)]

In [30]:
w2v_modelo.wv.most_similar("àcrak")

[('cocainda', 0.9451239109039307),
 ('esmarelada', 0.9037715792655945),
 ('aferilçao', 0.8924482464790344),
 ('qaurenta', 0.8648549318313599),
 ('toralizando', 0.8567660450935364),
 ('cocaica', 0.8491693735122681),
 ('caracteeisticas', 0.8359031081199646),
 ('commcaracteristica', 0.8236740827560425),
 ('flavia', 0.816604733467102),
 ('maconhanhah', 0.8155418634414673)]

In [37]:
#w2v_modelo_sg.wv.save_word2vec_format('../Models/modelo_sg_drogas.txt', binary=False)
#w2v_modelo.wv.save_word2vec_format('../Models/modelo_w2v_drogas.txt', binary=False)

2021-09-26 00:51:01,987 : - storing 8018x300 projection weights into ../Models/modelo_sg_drogas.txt
2021-09-26 00:51:04,108 : - storing 8018x300 projection weights into ../Models/modelo_w2v_drogas.txt


In [32]:
nlp = spacy.load('pt_core_news_lg', disable=['parser','ner','tagger','textcat'])

def tokenizador(texto):
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    return tokens_validos

texto = "42 gramas de maconha apreendidos"
tokens = tokenizador(texto)
print(tokens)

['gramas', 'maconha', 'apreendidos']


In [33]:
import numpy as np
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format('../Models/modelo_w2v_drogas.txt')
w2v_modelo_sg = KeyedVectors.load_word2vec_format('../Models/modelo_sg_drogas.txt')

2021-09-30 15:32:30,633 : - loading projection weights from ../Models/modelo_w2v_drogas.txt
2021-09-30 15:32:32,759 : - KeyedVectors lifecycle event {'msg': 'loaded (8018, 300) matrix of type float32 from ../Models/modelo_w2v_drogas.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-09-30T15:32:32.759350', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0', 'event': 'load_word2vec_format'}
2021-09-30 15:32:32,760 : - loading projection weights from ../Models/modelo_sg_drogas.txt
2021-09-30 15:32:34,713 : - KeyedVectors lifecycle event {'msg': 'loaded (8018, 300) matrix of type float32 from ../Models/modelo_sg_drogas.txt', 'binary': False, 'encoding': 'utf8', 'datetime': '2021-09-30T15:32:34.713352', 'gensim': '4.1.2', 'python': '3.9.7 | packaged by conda-forge | (default, Sep  2 2021, 17:55:20) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19042-SP0

In [34]:
def combinacao_de_vetores_por_soma(palavras, modelo):
    
    vetor_resultante = np.zeros((1, 300))
    
    for p in palavras:    
        try:
            vetor_resultante += modelo.get_vector(p)

        except KeyError:
           pass

    return vetor_resultante


vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[ 0.1286815  -1.90977759  0.13999526  1.59192276 -0.75647955 -0.01888183
   0.09412077 -0.1529047  -0.17223457  0.0638263  -0.10664336 -0.7182117
  -0.54101728 -0.59995277 -0.22309858  0.45804346 -0.06492934 -0.66322355
  -0.02955554  0.446263    1.27539738  1.01556657 -1.0327418   0.23260039
   1.17448529 -1.13782222  1.96738273  0.17222127  0.85432186 -0.08431077
   0.36445507  1.25217181  0.28909496 -0.81386827 -0.44321911 -0.87905075
   0.85559136 -1.61212209 -0.21839337 -0.31124746 -0.12727931 -0.31963331
   1.21662232  0.37635526  0.59015531  0.92955929 -1.36795782  0.10052644
   0.21649069 -0.64006384  0.45780528  0.59258001  0.2170861   0.45159146
  -0.76111618 -0.24482659  0.12455752 -0.57386573 -0.77144793  0.06450007
   0.4309913   0.28348443  0.09103077 -2.0773297  -0.12554504 -0.07390217
  -0.80918305 -0.30200598  0.28753987 -0.44278804  1.19332951 -1.11024354
  -0.89336275 -0.75083446  0.70570676 -0.1603111  -0.12427431  0.53098689
  -0.00523202 -0.04001921  1.5

In [35]:
def matriz_vetores(textos, modelo):
    #Numero de linhas da matriz depende do numero de linhas que eu quero vetorizar
    x = len(textos)
    #Numero de dimensoes
    y = 300
    matriz = np.zeros((x,y))

    #Vetorizar meu texto
    for i in range(x):
        palavras = tokenizador(str(textos.iloc[i]))
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz


tamanho_treino = len(titulos_tratados) * 0.8
tamanho_teste = len(titulos_tratados) * 0.2

matriz_vetores_treino_cbow = matriz_vetores(titulos_tratados.head(int(tamanho_treino)), w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(titulos_tratados.tail(int(tamanho_teste)), w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(41024, 300)
(10256, 300)


In [36]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [37]:
def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter  = 800)
    RL.fit(x_treino, y_treino)
    
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    
    print(resultados)
    return RL

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

frases =['Assisti um filme ótimo', 'Assisti um  filme péssimo']

tfidf = TfidfVectorizer(lowercase=False, max_features=50)

caracteristicas = tfidf.fit_transform(frases)
pd.DataFrame(
        caracteristicas.todense(),
        columns =  tfidf.get_feature_names()
)

,Assisti,filme,péssimo,um,ótimo
0,0.448321,0.448321,0.000000,0.448321,0.630099
1,0.448321,0.448321,0.630099,0.448321,0.000000


In [39]:
dados_treino_not_na = dados_treino.dropna()
dados_treino_drop_dup = dados_treino_not_na.drop_duplicates()
dados_treino_drop_dup.shape

(31209, 14)

In [40]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

regressaao_logistica1 = LogisticRegression(max_iter=100000)

vetor_tf_idf_bruto =  tfidf.fit_transform(dados_treino_drop_dup.DESC_PADRAO)
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tf_idf_bruto, 
                                                               dados_treino_drop_dup.RECLASSIFICAÇÃO,
                                                               random_state=42)


regressaao_logistica1.fit(treino,classe_treino)
acuraria_vetor_tf_idf_bruto = regressaao_logistica1.score(teste, classe_teste)
print(acuraria_vetor_tf_idf_bruto)

0.9416890939382289


In [41]:
from nltk import ngrams

tfidf = TfidfVectorizer(lowercase=False, ngram_range  = (1,2))
vetor_tfidf = tfidf.fit_transform(dados_treino_drop_dup.DESC_PADRAO)
treino, teste, classe_treino, classe_teste = train_test_split(vetor_tfidf, 
                                                               dados_treino_drop_dup.RECLASSIFICAÇÃO,
                                                               random_state=42)

regressaao_logistica2 = LogisticRegression(max_iter=100000)


regressaao_logistica2.fit(treino, classe_treino)
acuraria_vetor_tf_idf_ngrams = regressaao_logistica2.score(teste, classe_teste)
print(acuraria_vetor_tf_idf_ngrams)

0.9443803665256952


In [42]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[0].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
cocaina,16.588675
de cocaina,7.796512
cocaina pesando,5.226806
semelhante cocaina,5.017456
pinos,2.949326
po,2.709065
bucha,2.023448
buchas,1.960289
cocaina com,1.862976
cocaina peso,1.831586


In [43]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[1].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
crack,15.774618
de crack,7.475729
crack pesando,4.870457
semelhante crack,4.548877
pedras,4.018940
pedra,3.082007
crak,2.610897
pedras de,2.120847
pedrinhas,1.885873
ao crack,1.709113


In [44]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[3].T,
    index =  tfidf.get_feature_names(),
)

pesos.nlargest(10,0)

,0
maconha,16.865359
de maconha,9.161756
semelhante maconha,5.900847
maconha pesando,5.822899
cigarro,2.596435
maconha com,2.201777
tijolo,2.072743
maconha peso,2.042365
sativa,2.020073
similar maconha,1.814834


In [45]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[3].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
cocaina,-8.314541
crack,-7.257624
de cocaina,-3.749380
de crack,-3.358985
pedras,-2.919711
po,-2.895967
cocaina pesando,-2.568571
semelhante cocaina,-2.557396
pinos,-2.288249
crack pesando,-2.284207


In [46]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[2].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-5.950362
cocaina,-4.749549
crack,-3.846809
de maconha,-3.264561
de cocaina,-2.282599
semelhante maconha,-2.190798
de crack,-2.046092
maconha pesando,-1.943689
aprox,-1.817330
de,-1.629645


In [47]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[1].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-4.819093
cocaina,-3.524585
de maconha,-2.720131
semelhante maconha,-1.863405
maconha pesando,-1.785699
de cocaina,-1.764533
esverdeada,-1.382249
cigarro,-1.301423
erva,-1.266108
po,-1.255079


In [48]:
pesos = pd.DataFrame(
    regressaao_logistica2.coef_[0].T,
    index =  tfidf.get_feature_names(),
)

pesos.nsmallest(10,0)

,0
maconha,-6.095904
crack,-4.670186
de maconha,-3.177064
maconha pesando,-2.093511
de crack,-2.070652
semelhante maconha,-1.846644
esverdeada,-1.834838
erva,-1.692627
pedras,-1.643292
cigarro,-1.634815
